In [137]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
# Esto es una prueba

# time settings
dt = 0.01
steps = 100 # number of time steps
t = np.arange(0, dt*steps+0.01, dt) # time vector

# simulation settings
########################################################
l = 6               # meters
Area = 1            # m2
QL = 0.6            # m3/min
Yin = 20            # mg/L
QGin = 0.1          # N.m3/min
T = 283             # K
n = 6               # number of layers
p = 1.5
q = 0.9
r = 0.6
layers = range(0,6) 

########################################################

# some parameters calculated from simulation settings
#######################################################
VT = l*Area
Phi = 0.093*np.power((QGin/Area),0.9)
sg = 0.0
# Calculation of VGN
for i in layers:
    sg = sg + 1/(1+(l/n)*(i+1-0.5)/10.24)
VGN = Phi*VT/sg

# Calculation of VG1-VG6
VG = np.zeros(n)
for i in layers:
    VG[i] = VGN/(1+(l/n)*(i+1-0.5)/10.24)

# Calculation of QG1-QG6
QG = np.zeros(n)
for i in layers:
    QG[i] = QGin/(1+(l/n)*(i+1-1)/10.24)
    
# Calculation of VL1-VL6
VL = np.zeros(n)
for i in layers:
    VL[i] = VT/n - VG[i]
    
# Calculation of KLaN -> Eq. 9-32 from page 168 
KLaN = p*np.power((QGin/Area),q)

# Calculation of KLa
KLa = np.zeros(n)
for i in layers:
    term = (1+(l/n)*(i+1-0.5)/10.24)
    pw = -1*(2.0/3.0)
    KLa[i] = KLaN*np.power(term,pw)
    
# Other parameters
k3 = 0.017
kA = 0.0
kB = 0.0
alpha = 0.0
b = 0.0
temp_exp = 25.0
tmp = 273.15
ph = 7.0

m = 1.187*np.power(10.0,-7)*np.power(np.power(10.0,ph-14),-0.035)*tmp*np.exp(2428/tmp)
kp = 0.0
Pin = 0.0
#######################################################

#print(layers[-1])

X = np.zeros(n)
Y = np.zeros(n)
Ca = np.zeros(n)
Cb = np.zeros(n)
P = np.zeros(n)

def reactions(X, Y):
    Yk = np.zeros(n)
    Xk = np.zeros(n)
    for i in layers:
        if i == layers[0]:
            # Gas phase:
            Yk[i] = (QG[1]*Y[1] - QG[0]*Y[0] - VL[0]*KLa[0]*(m*Y[0] - X[0]))/VG[0]
            # Liquid phase:
            Xk[i] = (QL*(1+r)*(-1.0)*X[0] + r*QL*X[1] + VL[0]*KLa[0]*(m*Y[0] - X[0]) - 
                     VL[0]*(k3*X[0] - b*(-1.0*alpha*kA*Ca[0]*X[0] - kB*Cb[0]*X[0] - kp*X[0]*P[0])))/VL[0]
            # Los C's
            
        
        elif i == layers[-1]:
            # Gas phase:
            Yk[i] = (QGin*Yin - QG[-1]*Y[-1] - VL[-1]*KLa[-1]*(m*Y[-1] - X[-1]))/VG[-1]
            # Liquid phase:
            Xk[i] = (QL*(1+r)*X[-2] + QL*(1+r)*(-1.0)*X[-1] + VL[-1]*KLa[-1]*(m*Y[-1]-X[-1]) - 
                    VL[-1]*(k3*X[-1] - b*(-1.0*alpha*kA*Ca[-1]*X[-1] - kB*Cb[-1]*X[-1] - kp*X[-1]*P[-1])))/VL[-1]
        else:
            # Gas phase:
            Yk[i] = (QG[i+1]*Y[i+1] - QG[i]*Y[i] - VL[i]*KLa[i]*(m*Y[i] - X[i]))/VG[i]
            # Liquid phase
            Xk[i] = (QL*(1+r)*(X[i-1]-X[i]) + r*QL*(X[i+1]-X[i]) + VL[i]*KLa[i]*(m*Y[i]-X[i]) - 
                    VL[i]*(k3*X[i] - b*(-1.0*alpha*kA*Ca[i]*X[i] - kB*Cb[i]*X[i] - kp*X[i]*P[i])))/VL[i]
    
    arg = (Yk,Xk)
    k = np.concatenate(arg)
    #print(k)
    return(k)



shoki = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

Concentrations = odeint(reactions,shoki,t)

#print(X)

TypeError: 'float' object is not subscriptable